In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import LearningRateScheduler,ModelCheckpoint
from skimage import color, exposure, transform,io
import numpy as np
import os
import glob
import h5py
from keras.optimizers import SGD
from multiprocessing import Process, Queue
import multiprocessing
NUM_CLASSES = 11
IMG_SIZE = 224

Using TensorFlow backend.


In [2]:
batch_size = 16
epochs = 150

In [3]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))


In [4]:
def preprocess_img(img):
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    # central square crop
    min_side = min(img.shape[:-1])
    centre = img.shape[0] // 2, img.shape[1] // 2
    img = img[centre[0] - min_side // 2:centre[0] + min_side // 2,
              centre[1] - min_side // 2:centre[1] + min_side // 2,
              :]

    # rescale to standard size
    img = transform.resize(img, (480, 270))

    # roll color axis to axis 0
    img = np.rollaxis(img, -1)

    return img

In [5]:
def get_class(img_path):
    return int(img_path.split('/')[-2])

root_dir = '/home/dae/workspace/music/8class/new/'
imgs = []
labels = []

all_img_paths = glob.glob(os.path.join(root_dir, '*/*.jpg'))
np.random.shuffle(all_img_paths)

In [6]:
len(all_img_paths)//multiprocessing.cpu_count()

1893

In [7]:
#def pre_process(start,end,imgs,labels):
#    for i in range(start,end):
#        img = preprocess_img(io.imread(all_img_paths[i]))
#        label = get_class(all_img_paths[i])
#        imgs.append(img)
#        labels.append(label)

In [8]:
img = preprocess_img(io.imread(all_img_paths[0]))

/home/dae/anaconda3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
length = len(all_img_paths)

In [10]:
i = 0;
for img_path in all_img_paths:
    img = preprocess_img(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)
    i=i+1
    print("%d / %d " %(i,length))
    

X = np.array(imgs, dtype='float32')
# Make one hot targets
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

/home/dae/anaconda3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


1 / 15145 
2 / 15145 
3 / 15145 
4 / 15145 
5 / 15145 
6 / 15145 
7 / 15145 
8 / 15145 
9 / 15145 
10 / 15145 
11 / 15145 
12 / 15145 
13 / 15145 
14 / 15145 
15 / 15145 
16 / 15145 
17 / 15145 
18 / 15145 
19 / 15145 
20 / 15145 
21 / 15145 
22 / 15145 
23 / 15145 
24 / 15145 
25 / 15145 
26 / 15145 
27 / 15145 
28 / 15145 
29 / 15145 
30 / 15145 
31 / 15145 
32 / 15145 
33 / 15145 
34 / 15145 
35 / 15145 
36 / 15145 
37 / 15145 
38 / 15145 
39 / 15145 
40 / 15145 
41 / 15145 
42 / 15145 
43 / 15145 
44 / 15145 
45 / 15145 
46 / 15145 
47 / 15145 
48 / 15145 
49 / 15145 
50 / 15145 
51 / 15145 
52 / 15145 
53 / 15145 
54 / 15145 
55 / 15145 
56 / 15145 
57 / 15145 
58 / 15145 
59 / 15145 
60 / 15145 
61 / 15145 
62 / 15145 
63 / 15145 
64 / 15145 
65 / 15145 
66 / 15145 
67 / 15145 
68 / 15145 
69 / 15145 
70 / 15145 
71 / 15145 
72 / 15145 
73 / 15145 
74 / 15145 
75 / 15145 
76 / 15145 
77 / 15145 
78 / 15145 
79 / 15145 
80 / 15145 
81 / 15145 
82 / 15145 
83 / 15145 
84 / 15145 
8

640 / 15145 
641 / 15145 
642 / 15145 
643 / 15145 
644 / 15145 
645 / 15145 
646 / 15145 
647 / 15145 
648 / 15145 
649 / 15145 
650 / 15145 
651 / 15145 
652 / 15145 
653 / 15145 
654 / 15145 
655 / 15145 
656 / 15145 
657 / 15145 
658 / 15145 
659 / 15145 
660 / 15145 
661 / 15145 
662 / 15145 
663 / 15145 
664 / 15145 
665 / 15145 
666 / 15145 
667 / 15145 
668 / 15145 
669 / 15145 
670 / 15145 
671 / 15145 
672 / 15145 
673 / 15145 
674 / 15145 
675 / 15145 
676 / 15145 
677 / 15145 
678 / 15145 
679 / 15145 
680 / 15145 
681 / 15145 
682 / 15145 
683 / 15145 
684 / 15145 
685 / 15145 
686 / 15145 
687 / 15145 
688 / 15145 
689 / 15145 
690 / 15145 
691 / 15145 
692 / 15145 
693 / 15145 
694 / 15145 
695 / 15145 
696 / 15145 
697 / 15145 
698 / 15145 
699 / 15145 
700 / 15145 
701 / 15145 
702 / 15145 
703 / 15145 
704 / 15145 
705 / 15145 
706 / 15145 
707 / 15145 
708 / 15145 
709 / 15145 
710 / 15145 
711 / 15145 
712 / 15145 
713 / 15145 
714 / 15145 
715 / 15145 
716 / 15145 

1251 / 15145 
1252 / 15145 
1253 / 15145 
1254 / 15145 
1255 / 15145 
1256 / 15145 
1257 / 15145 
1258 / 15145 
1259 / 15145 
1260 / 15145 
1261 / 15145 
1262 / 15145 
1263 / 15145 
1264 / 15145 
1265 / 15145 
1266 / 15145 
1267 / 15145 
1268 / 15145 
1269 / 15145 
1270 / 15145 
1271 / 15145 
1272 / 15145 
1273 / 15145 
1274 / 15145 
1275 / 15145 
1276 / 15145 
1277 / 15145 
1278 / 15145 
1279 / 15145 
1280 / 15145 
1281 / 15145 
1282 / 15145 
1283 / 15145 
1284 / 15145 
1285 / 15145 
1286 / 15145 
1287 / 15145 
1288 / 15145 
1289 / 15145 
1290 / 15145 
1291 / 15145 
1292 / 15145 
1293 / 15145 
1294 / 15145 
1295 / 15145 
1296 / 15145 
1297 / 15145 
1298 / 15145 
1299 / 15145 
1300 / 15145 
1301 / 15145 
1302 / 15145 
1303 / 15145 
1304 / 15145 
1305 / 15145 
1306 / 15145 
1307 / 15145 
1308 / 15145 
1309 / 15145 
1310 / 15145 
1311 / 15145 
1312 / 15145 
1313 / 15145 
1314 / 15145 
1315 / 15145 
1316 / 15145 
1317 / 15145 
1318 / 15145 
1319 / 15145 
1320 / 15145 
1321 / 15145 
1322 /

1837 / 15145 
1838 / 15145 
1839 / 15145 
1840 / 15145 
1841 / 15145 
1842 / 15145 
1843 / 15145 
1844 / 15145 
1845 / 15145 
1846 / 15145 
1847 / 15145 
1848 / 15145 
1849 / 15145 
1850 / 15145 
1851 / 15145 
1852 / 15145 
1853 / 15145 
1854 / 15145 
1855 / 15145 
1856 / 15145 
1857 / 15145 
1858 / 15145 
1859 / 15145 
1860 / 15145 
1861 / 15145 
1862 / 15145 
1863 / 15145 
1864 / 15145 
1865 / 15145 
1866 / 15145 
1867 / 15145 
1868 / 15145 
1869 / 15145 
1870 / 15145 
1871 / 15145 
1872 / 15145 
1873 / 15145 
1874 / 15145 
1875 / 15145 
1876 / 15145 
1877 / 15145 
1878 / 15145 
1879 / 15145 
1880 / 15145 
1881 / 15145 
1882 / 15145 
1883 / 15145 
1884 / 15145 
1885 / 15145 
1886 / 15145 
1887 / 15145 
1888 / 15145 
1889 / 15145 
1890 / 15145 
1891 / 15145 
1892 / 15145 
1893 / 15145 
1894 / 15145 
1895 / 15145 
1896 / 15145 
1897 / 15145 
1898 / 15145 
1899 / 15145 
1900 / 15145 
1901 / 15145 
1902 / 15145 
1903 / 15145 
1904 / 15145 
1905 / 15145 
1906 / 15145 
1907 / 15145 
1908 /

...

12773 / 15145 
12774 / 15145 
12775 / 15145 
12776 / 15145 
12777 / 15145 
12778 / 15145 
12779 / 15145 
12780 / 15145 
12781 / 15145 
12782 / 15145 
12783 / 15145 
12784 / 15145 
12785 / 15145 
12786 / 15145 
12787 / 15145 
12788 / 15145 
12789 / 15145 
12790 / 15145 
12791 / 15145 
12792 / 15145 
12793 / 15145 
12794 / 15145 
12795 / 15145 
12796 / 15145 
12797 / 15145 
12798 / 15145 
12799 / 15145 
12800 / 15145 
12801 / 15145 
12802 / 15145 
12803 / 15145 
12804 / 15145 
12805 / 15145 
12806 / 15145 
12807 / 15145 
12808 / 15145 
12809 / 15145 
12810 / 15145 
12811 / 15145 
12812 / 15145 
12813 / 15145 
12814 / 15145 
12815 / 15145 
12816 / 15145 
12817 / 15145 
12818 / 15145 
12819 / 15145 
12820 / 15145 
12821 / 15145 
12822 / 15145 
12823 / 15145 
12824 / 15145 
12825 / 15145 
12826 / 15145 
12827 / 15145 
12828 / 15145 
12829 / 15145 
12830 / 15145 
12831 / 15145 
12832 / 15145 
12833 / 15145 
12834 / 15145 
12835 / 15145 
12836 / 15145 
12837 / 15145 
12838 / 15145 
12839 / 15

13320 / 15145 
13321 / 15145 
13322 / 15145 
13323 / 15145 
13324 / 15145 
13325 / 15145 
13326 / 15145 
13327 / 15145 
13328 / 15145 
13329 / 15145 
13330 / 15145 
13331 / 15145 
13332 / 15145 
13333 / 15145 
13334 / 15145 
13335 / 15145 
13336 / 15145 
13337 / 15145 
13338 / 15145 
13339 / 15145 
13340 / 15145 
13341 / 15145 
13342 / 15145 
13343 / 15145 
13344 / 15145 
13345 / 15145 
13346 / 15145 
13347 / 15145 
13348 / 15145 
13349 / 15145 
13350 / 15145 
13351 / 15145 
13352 / 15145 
13353 / 15145 
13354 / 15145 
13355 / 15145 
13356 / 15145 
13357 / 15145 
13358 / 15145 
13359 / 15145 
13360 / 15145 
13361 / 15145 
13362 / 15145 
13363 / 15145 
13364 / 15145 
13365 / 15145 
13366 / 15145 
13367 / 15145 
13368 / 15145 
13369 / 15145 
13370 / 15145 
13371 / 15145 
13372 / 15145 
13373 / 15145 
13374 / 15145 
13375 / 15145 
13376 / 15145 
13377 / 15145 
13378 / 15145 
13379 / 15145 
13380 / 15145 
13381 / 15145 
13382 / 15145 
13383 / 15145 
13384 / 15145 
13385 / 15145 
13386 / 15

13867 / 15145 
13868 / 15145 
13869 / 15145 
13870 / 15145 
13871 / 15145 
13872 / 15145 
13873 / 15145 
13874 / 15145 
13875 / 15145 
13876 / 15145 
13877 / 15145 
13878 / 15145 
13879 / 15145 
13880 / 15145 
13881 / 15145 
13882 / 15145 
13883 / 15145 
13884 / 15145 
13885 / 15145 
13886 / 15145 
13887 / 15145 
13888 / 15145 
13889 / 15145 
13890 / 15145 
13891 / 15145 
13892 / 15145 
13893 / 15145 
13894 / 15145 
13895 / 15145 
13896 / 15145 
13897 / 15145 
13898 / 15145 
13899 / 15145 
13900 / 15145 
13901 / 15145 
13902 / 15145 
13903 / 15145 
13904 / 15145 
13905 / 15145 
13906 / 15145 
13907 / 15145 
13908 / 15145 
13909 / 15145 
13910 / 15145 
13911 / 15145 
13912 / 15145 
13913 / 15145 
13914 / 15145 
13915 / 15145 
13916 / 15145 
13917 / 15145 
13918 / 15145 
13919 / 15145 
13920 / 15145 
13921 / 15145 
13922 / 15145 
13923 / 15145 
13924 / 15145 
13925 / 15145 
13926 / 15145 
13927 / 15145 
13928 / 15145 
13929 / 15145 
13930 / 15145 
13931 / 15145 
13932 / 15145 
13933 / 15

14414 / 15145 
14415 / 15145 
14416 / 15145 
14417 / 15145 
14418 / 15145 
14419 / 15145 
14420 / 15145 
14421 / 15145 
14422 / 15145 
14423 / 15145 
14424 / 15145 
14425 / 15145 
14426 / 15145 
14427 / 15145 
14428 / 15145 
14429 / 15145 
14430 / 15145 
14431 / 15145 
14432 / 15145 
14433 / 15145 
14434 / 15145 
14435 / 15145 
14436 / 15145 
14437 / 15145 
14438 / 15145 
14439 / 15145 
14440 / 15145 
14441 / 15145 
14442 / 15145 
14443 / 15145 
14444 / 15145 
14445 / 15145 
14446 / 15145 
14447 / 15145 
14448 / 15145 
14449 / 15145 
14450 / 15145 
14451 / 15145 
14452 / 15145 
14453 / 15145 
14454 / 15145 
14455 / 15145 
14456 / 15145 
14457 / 15145 
14458 / 15145 
14459 / 15145 
14460 / 15145 
14461 / 15145 
14462 / 15145 
14463 / 15145 
14464 / 15145 
14465 / 15145 
14466 / 15145 
14467 / 15145 
14468 / 15145 
14469 / 15145 
14470 / 15145 
14471 / 15145 
14472 / 15145 
14473 / 15145 
14474 / 15145 
14475 / 15145 
14476 / 15145 
14477 / 15145 
14478 / 15145 
14479 / 15145 
14480 / 15

14961 / 15145 
14962 / 15145 
14963 / 15145 
14964 / 15145 
14965 / 15145 
14966 / 15145 
14967 / 15145 
14968 / 15145 
14969 / 15145 
14970 / 15145 
14971 / 15145 
14972 / 15145 
14973 / 15145 
14974 / 15145 
14975 / 15145 
14976 / 15145 
14977 / 15145 
14978 / 15145 
14979 / 15145 
14980 / 15145 
14981 / 15145 
14982 / 15145 
14983 / 15145 
14984 / 15145 
14985 / 15145 
14986 / 15145 
14987 / 15145 
14988 / 15145 
14989 / 15145 
14990 / 15145 
14991 / 15145 
14992 / 15145 
14993 / 15145 
14994 / 15145 
14995 / 15145 
14996 / 15145 
14997 / 15145 
14998 / 15145 
14999 / 15145 
15000 / 15145 
15001 / 15145 
15002 / 15145 
15003 / 15145 
15004 / 15145 
15005 / 15145 
15006 / 15145 
15007 / 15145 
15008 / 15145 
15009 / 15145 
15010 / 15145 
15011 / 15145 
15012 / 15145 
15013 / 15145 
15014 / 15145 
15015 / 15145 
15016 / 15145 
15017 / 15145 
15018 / 15145 
15019 / 15145 
15020 / 15145 
15021 / 15145 
15022 / 15145 
15023 / 15145 
15024 / 15145 
15025 / 15145 
15026 / 15145 
15027 / 15

In [20]:
X = np.array(imgs, dtype='float32')
# Make one hot targets
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

In [ ]:
root_dir = '/home/dae/workspace/music/test8'
imgs_t = []
labels_t = []

all_img_paths_t = glob.glob(os.path.join(root_dir, '*/*.jpg'))
np.random.shuffle(all_img_paths_t)

for img_path in all_img_paths_t:
    img = preprocess_img(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)
 

X_t = np.array(imgs, dtype='float32')
# Make one hot targets
Y_t = np.eye(NUM_CLASSES, dtype='uint8')[labels]


In [11]:
K.set_image_data_format('channels_first')
model = Sequential()
model.add(Conv2D(32, (11, 7), padding='same',
                 input_shape=(3, 480, 270),
                 activation='relu'))
model.add(Conv2D(32, (11, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (9, 5), padding='same',
                 activation='relu'))
model.add(Conv2D(64, (9, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), padding='same',
                 activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 480, 270)      7424      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 470, 264)      78880     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 156, 88)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 156, 88)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 156, 88)       92224     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 148, 84)       184384    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 49, 28)        0         
__________

In [7]:
from keras.models import model_from_json

In [8]:

json_string = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(json_string)

In [7]:
from keras.models import load_model
model = load_model('modeldec.h5')

In [10]:
arr = model.layers[-1].get_weights()[0]

In [11]:
arr

array([[-0.06900068, -0.04566348, -0.05623586, ...,  0.08505024,
        -0.06882077, -0.06267742],
       [ 0.00294334, -0.04573676, -0.04421797, ..., -0.06628606,
        -0.04405116,  0.0894144 ],
       [ 0.00911912, -0.01880526, -0.05158838, ..., -0.01455999,
        -0.01045216, -0.0243484 ],
       ..., 
       [ 0.05248804, -0.04041849, -0.04981874, ...,  0.09012336,
        -0.10074599, -0.01433135],
       [-0.08812528,  0.18597345, -0.1728209 , ...,  0.02574006,
        -0.05524066,  0.1002745 ],
       [ 0.01007402,  0.02996697, -0.05944567, ...,  0.0363843 ,
         0.05762884, -0.01009594]], dtype=float32)

In [13]:
len(arr)

512

In [91]:
X = np.array(arr)

In [96]:
X[1][1]

-0.062961966

In [50]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 224, 224)      7424      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 214, 218)      78880     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 107, 72)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 107, 72)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 107, 72)       43072     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 101, 70)       86080     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 50, 23)        0         
__________

In [35]:
X = np.array(arr)

(7, 3, 32)

In [47]:
K.clear_session()
model = None
del model

In [12]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          callbacks=[LearningRateScheduler(lr_schedule)
                     ,ModelCheckpoint('modeldec_modifi.h5', save_best_only=True)]
          )


Train on 13630 samples, validate on 1515 samples
Epoch 1/150
13630/13630 [==============================] - 594s - loss: 2.0913 - acc: 0.2894 - val_loss: 2.0370 - val_acc: 0.2851
Epoch 2/150
13630/13630 [==============================] - 497s - loss: 1.9519 - acc: 0.3347 - val_loss: 1.8754 - val_acc: 0.3703
Epoch 3/150
13630/13630 [==============================] - 484s - loss: 1.8640 - acc: 0.3624 - val_loss: 1.8727 - val_acc: 0.3498
Epoch 4/150
13630/13630 [==============================] - 483s - loss: 1.8240 - acc: 0.3745 - val_loss: 1.8404 - val_acc: 0.3729
Epoch 5/150
13630/13630 [==============================] - 483s - loss: 1.8620 - acc: 0.3577 - val_loss: 1.7924 - val_acc: 0.3927
Epoch 6/150
13630/13630 [==============================] - 484s - loss: 1.8224 - acc: 0.3737 - val_loss: 1.9365 - val_acc: 0.3340
Epoch 7/150
13630/13630 [==============================] - 484s - loss: 1.8196 - acc: 0.3743 - val_loss: 1.8285 - val_acc: 0.3894
Epoch 8/150
13630/13630 [================

13630/13630 [==============================] - 481s - loss: 1.2939 - acc: 0.5579 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 64/150
13630/13630 [==============================] - 481s - loss: 1.2858 - acc: 0.5583 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 65/150
13630/13630 [==============================] - 481s - loss: 1.2904 - acc: 0.5567 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 66/150
13630/13630 [==============================] - 481s - loss: 1.2882 - acc: 0.5574 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 67/150
13630/13630 [==============================] - 481s - loss: 1.2890 - acc: 0.5596 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 68/150
13630/13630 [==============================] - 481s - loss: 1.2856 - acc: 0.5585 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 69/150
13630/13630 [==============================] - 481s - loss: 1.2875 - acc: 0.5552 - val_loss: 1.5627 - val_acc: 0.4818
Epoch 70/150
13630/13630 [==============================] - 481s - loss: 1.2891 - acc: 0.5574 - 

13630/13630 [==============================] - 481s - loss: 1.2875 - acc: 0.5602 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 126/150
13630/13630 [==============================] - 481s - loss: 1.2949 - acc: 0.5514 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 127/150
13630/13630 [==============================] - 481s - loss: 1.2887 - acc: 0.5591 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 128/150
13630/13630 [==============================] - 481s - loss: 1.2874 - acc: 0.5606 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 129/150
13630/13630 [==============================] - 481s - loss: 1.2930 - acc: 0.5583 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 130/150
13630/13630 [==============================] - 481s - loss: 1.2917 - acc: 0.5577 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 131/150
13630/13630 [==============================] - 481s - loss: 1.2946 - acc: 0.5573 - val_loss: 1.5626 - val_acc: 0.4818
Epoch 132/150
13630/13630 [==============================] - 481s - loss: 1.2852 - acc: 0.

In [14]:
y_pred = model.predict(X_t)
for i in range(y_pred.shape[0]):
       for j in range(5):
            tmp = max(y_pred[i])
            if( y_pred[i][j] == tmp) :
                y_pred[i][j] = 1
            elif (y_pred[i][j] != tmp):
                y_pred[i][j] = 0

acc = np.sum(y_pred == Y_t) / np.size(X_t)
print("Test accuracy = {}".format(acc))

Test accuracy = 3.1151117964378067e-05


In [13]:
K.set_image_data_format('channels_first')
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(3, 480, 270),
                 activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.4))

model.add(Conv2D(64, (3, 3), padding='same',
                 activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same',
                 activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 480, 270)      896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 478, 268)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 159, 89)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 159, 89)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 159, 89)       18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 157, 87)       36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 52, 29)        0         
__________

In [14]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 16
epochs = 200

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          callbacks=[LearningRateScheduler(lr_schedule)
                     ,ModelCheckpoint('model117_new.h5', save_best_only=True)]
          )


Train on 13630 samples, validate on 1515 samples
Epoch 1/200
13630/13630 [==============================] - 405s - loss: 1.8431 - acc: 0.3763 - val_loss: 1.6392 - val_acc: 0.3657
Epoch 2/200
13630/13630 [==============================] - 404s - loss: 1.2957 - acc: 0.5542 - val_loss: 1.3297 - val_acc: 0.5162
Epoch 3/200
13630/13630 [==============================] - 404s - loss: 1.1403 - acc: 0.5929 - val_loss: 1.1064 - val_acc: 0.5881
Epoch 4/200
13630/13630 [==============================] - 404s - loss: 1.0635 - acc: 0.6254 - val_loss: 0.9322 - val_acc: 0.6554
Epoch 5/200
13630/13630 [==============================] - 404s - loss: 1.0117 - acc: 0.6394 - val_loss: 0.9210 - val_acc: 0.6799
Epoch 6/200
13630/13630 [==============================] - 404s - loss: 0.9386 - acc: 0.6712 - val_loss: 0.8687 - val_acc: 0.6911
Epoch 7/200
13630/13630 [==============================] - 404s - loss: 0.8963 - acc: 0.6871 - val_loss: 0.8994 - val_acc: 0.6871
Epoch 8/200
13630/13630 [================

13630/13630 [==============================] - 404s - loss: 0.3579 - acc: 0.8704 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 64/200
13630/13630 [==============================] - 404s - loss: 0.3578 - acc: 0.8692 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 65/200
13630/13630 [==============================] - 404s - loss: 0.3559 - acc: 0.8740 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 66/200
13630/13630 [==============================] - 404s - loss: 0.3582 - acc: 0.8704 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 67/200
13630/13630 [==============================] - 404s - loss: 0.3594 - acc: 0.8655 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 68/200
13630/13630 [==============================] - 404s - loss: 0.3572 - acc: 0.8671 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 69/200
13630/13630 [==============================] - 404s - loss: 0.3603 - acc: 0.8689 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 70/200
13630/13630 [==============================] - 404s - loss: 0.3529 - acc: 0.8709 - 

13630/13630 [==============================] - 404s - loss: 0.3577 - acc: 0.8734 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 126/200
13630/13630 [==============================] - 404s - loss: 0.3529 - acc: 0.8727 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 127/200
13630/13630 [==============================] - 404s - loss: 0.3484 - acc: 0.8746 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 128/200
13630/13630 [==============================] - 404s - loss: 0.3530 - acc: 0.8723 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 129/200
13630/13630 [==============================] - 404s - loss: 0.3568 - acc: 0.8702 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 130/200
13630/13630 [==============================] - 404s - loss: 0.3647 - acc: 0.8679 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 131/200
13630/13630 [==============================] - 404s - loss: 0.3586 - acc: 0.8685 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 132/200
13630/13630 [==============================] - 404s - loss: 0.3588 - acc: 0.

13630/13630 [==============================] - 404s - loss: 0.3656 - acc: 0.8676 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 188/200
13630/13630 [==============================] - 404s - loss: 0.3559 - acc: 0.8698 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 189/200
13630/13630 [==============================] - 404s - loss: 0.3562 - acc: 0.8730 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 190/200
13630/13630 [==============================] - 404s - loss: 0.3529 - acc: 0.8718 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 191/200
13630/13630 [==============================] - 404s - loss: 0.3558 - acc: 0.8715 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 192/200
13630/13630 [==============================] - 404s - loss: 0.3560 - acc: 0.8705 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 193/200
13630/13630 [==============================] - 404s - loss: 0.3634 - acc: 0.8676 - val_loss: 0.6628 - val_acc: 0.7914
Epoch 194/200
13630/13630 [==============================] - 404s - loss: 0.3602 - acc: 0.

In [17]:
y_pred = model.predict(X_t)
for i in range(y_pred.shape[0]):
       for j in range(5):
            tmp = max(y_pred[i])
            if( y_pred[i][j] == tmp) :
                y_pred[i][j] = 1
            elif (y_pred[i][j] != tmp):
                y_pred[i][j] = 0

acc = np.sum(y_pred == Y_t) / np.size(X_t)
print("Test accuracy = {}".format(acc))

Test accuracy = 2.3132548608724968e-05


In [ ]:
K.set_image_data_format('channels_first')
model = Sequential()
model.add(Conv2D(32, (9, 5), padding='same',
                 input_shape=(3, IMG_SIZE, IMG_SIZE),
                 activation='relu'))
model.add(Conv2D(32, (9, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (9, 5), padding='same',
                 activation='relu'))
model.add(Conv2D(64, (9, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (9, 5), padding='same',
                 activation='relu'))
model.add(Conv2D(128, (9, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


In [ ]:
batch_size = 16
epochs = 200

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          callbacks=[LearningRateScheduler(lr_schedule)
                     ,ModelCheckpoint('model95.h5', save_best_only=True)]
          )


###### y_pred = model.predict(X_t)
for i in range(y_pred.shape[0]):
       for j in range(5):
            tmp = max(y_pred[i])
            if( y_pred[i][j] == tmp) :
                y_pred[i][j] = 1
            elif (y_pred[i][j] != tmp):
                y_pred[i][j] = 0

acc = np.sum(y_pred == Y_t) / np.size(X_t)
print("Test accuracy = {}".format(acc))


In [ ]:
K.set_image_data_format('channels_first')
model = Sequential()
model.add(Conv2D(32, (7, 3), padding='same',
                 input_shape=(3, IMG_SIZE, IMG_SIZE),
                 activation='relu'))
model.add(Conv2D(32, (7, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (7, 3), padding='same',
                 activation='relu'))
model.add(Conv2D(64, (7, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (7, 3), padding='same',
                 activation='relu'))
model.add(Conv2D(128, (7, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


In [ ]:
model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          callbacks=[LearningRateScheduler(lr_schedule)
                     ,ModelCheckpoint('model73.h5', save_best_only=True)]
          )


### y_pred = model.predict(X_t)
for i in range(y_pred.shape[0]):
       for j in range(5):
            tmp = max(y_pred[i])
            if( y_pred[i][j] == tmp) :
                y_pred[i][j] = 1
            elif (y_pred[i][j] != tmp):
                y_pred[i][j] = 0

acc = np.sum(y_pred == Y_t) / np.size(X_t)
print("Test accuracy = {}".format(acc))

In [24]:
X.shape

(7934, 3, 224, 224)

In [25]:
Y.shape

(7934, 5)

In [26]:
X

array([[[[  1.54517576e-01,   1.21340185e-01,   1.51258394e-01, ...,
            7.00269490e-02,   2.37656087e-01,   2.93358922e-01],
         [  1.39283329e-01,   1.51273623e-01,   1.78746879e-01, ...,
            2.93975443e-01,   3.08625907e-01,   3.96018326e-01],
         [  1.56857312e-01,   1.03609309e-01,   1.22445971e-01, ...,
            2.60278553e-01,   2.57462442e-01,   3.53757203e-01],
         ..., 
         [  1.12849966e-01,   1.00561827e-01,   1.62229594e-02, ...,
            5.48998237e-01,   4.98625517e-01,   5.03770769e-01],
         [  4.13624555e-01,   3.74890536e-01,   2.72072673e-01, ...,
            5.29495120e-01,   5.24644732e-01,   5.46334028e-01],
         [  1.79201353e-03,   1.75156596e-03,   1.95217715e-03, ...,
            2.04831199e-03,   2.03803764e-03,   2.23359582e-03]],

        [[  2.27002427e-02,   1.81419645e-02,   2.49569062e-02, ...,
            9.20346286e-03,   4.17999253e-02,   5.92636764e-02],
         [  1.93914715e-02,   2.26459038e-02,

In [32]:
y_pred = model.predict(X)
acc = np.sum(y_pred == Y) / np.size(y_pred)

In [48]:
print("Test accuracy = {}".format(acc))

Test accuracy = 0.004151667866570674


In [37]:
Y.shape

(8334, 5)

In [38]:
y_pred.shape

(8334, 5)

In [43]:
y_pred[0]

array([  1.09945416e-01,   7.70993647e-04,   8.89283597e-01,
         4.02227585e-09,   6.93631472e-11], dtype=float32)

In [51]:
X[1].shape

(3, 224, 224)

In [52]:
y_pred

array([[  1.09945416e-01,   7.70993647e-04,   8.89283597e-01,
          4.02227585e-09,   6.93631472e-11],
       [  8.66896689e-01,   1.01333171e-06,   1.33054793e-01,
          2.46588588e-05,   2.28104145e-05],
       [  9.85474110e-01,   4.33464884e-05,   1.44813107e-02,
          1.13987653e-06,   2.28670971e-09],
       ..., 
       [  9.75519717e-02,   7.00174689e-01,   1.94534976e-02,
          1.36453146e-02,   1.69174626e-01],
       [  5.55600040e-02,   1.53500459e-05,   9.31040406e-01,
          1.24795772e-02,   9.04629473e-04],
       [  9.99985456e-01,   8.90287447e-06,   2.86678642e-06,
          2.78997595e-06,   5.14177080e-13]], dtype=float32)

In [59]:
from keras.utils.np_utils import to_categorical

In [63]:
categorical_labels = to_categorical(y_pred,5)

In [85]:
y_pred.shape[0]

8334

In [86]:
for i in range(y_pred.shape[0]):
       for j in range(5):
            tmp = max(y_pred[i])
            if( y_pred[i][j] == tmp) :
                y_pred[i][j] = 1
            elif (y_pred[i][j] != tmp):
                y_pred[i][j] = 0


In [89]:
acc = np.sum(y_pred == Y) / np.size(y_pred)
print("Test accuracy = {}".format(acc))

Test accuracy = 0.9749940004799617


In [71]:
model.evaluate(X,Y)

8320/8334 [============================>.] - ETA: 0s

[0.18929317188056963, 0.93748500125711987]

In [72]:
A = model.predict(X)

In [73]:
A

array([[  1.09945416e-01,   7.70993647e-04,   8.89283597e-01,
          4.02227585e-09,   6.93631472e-11],
       [  8.66896689e-01,   1.01333171e-06,   1.33054793e-01,
          2.46588588e-05,   2.28104145e-05],
       [  9.85474110e-01,   4.33464884e-05,   1.44813107e-02,
          1.13987653e-06,   2.28670971e-09],
       ..., 
       [  9.75519717e-02,   7.00174689e-01,   1.94534976e-02,
          1.36453146e-02,   1.69174626e-01],
       [  5.55600040e-02,   1.53500459e-05,   9.31040406e-01,
          1.24795772e-02,   9.04629473e-04],
       [  9.99985456e-01,   8.90287447e-06,   2.86678642e-06,
          2.78997595e-06,   5.14177080e-13]], dtype=float32)

In [36]:
y_pred = model.predict(X_t)
for i in range(y_pred.shape[0]):
       for j in range(5):
            tmp = max(y_pred[i])
            if( y_pred[i][j] == tmp) :
                y_pred[i][j] = 1
            elif (y_pred[i][j] != tmp):
                y_pred[i][j] = 0

acc = np.sum(y_pred == Y_t) / np.size(y_pred)
print("Test accuracy = {}".format(acc))


Test accuracy = 0.9757647616789187


In [35]:
np.sum(y_pred[1]==Y_t[1])

5

In [33]:
np.size(X_t)

1269553152